##### - DNN기반 회귀 구현 + 학습 스케쥴링
 - 데이터셋 : IRIS.csv
 - 피쳐/속성 : 3개 Setal_Length, Sepal_Width, Petal_Width
 - 타겟/라벨 : 1개 Petal_Length
 - 학습-방법 : 지도학습 > 회귀
 - 알고 리즘 : 인공신경망(ANN) -> 심층신경망(MLP), DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch
 * * *
 - 모니터링
    * 시 동적으로 LR값을 조정해주는 

[1] 모듈 로딩과 데이터 준비 <hr>

In [89]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary
 
# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [90]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하기
print(f'torch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

torch v.2.4.1
Pandas v.2.0.3


In [91]:
print(dir())

['BATCH_CNT', 'BATCH_SIZE', 'BREAK_CNT', 'Binarizer', 'CNT', 'DATA_FILE', 'DEVICE', 'DataLoader', 'Dataset', 'EPOCH', 'F', 'FunctionTransformer', 'In', 'IrisDataset', 'IrisRegModel', 'KBinsDiscretizer', 'KernelCenterer', 'LOSS_HISTORY', 'LR', 'LabelBinarizer', 'LabelEncoder', 'MaxAbsScaler', 'MeanSquaredError', 'MinMaxScaler', 'MultiLabelBinarizer', 'Normalizer', 'OneHotEncoder', 'OrdinalEncoder', 'Out', 'PolynomialFeatures', 'PowerTransformer', 'QuantileTransformer', 'R2Score', 'RobustScaler', 'SAVE_FILE', 'SAVE_PATH', 'SCORE_HISTORY', 'SplineTransformer', 'StandardScaler', 'TargetEncoder', 'X_test', 'X_train', 'X_val', '_', '_19', '_22', '_27', '_37', '_40', '_45', '_49', '_52', '_57', '_65', '_68', '_73', '_77', '_80', '_85', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', 

In [92]:
def ver_check():
    import_list = list(dir())
    for i in import_list:
        try:
            
            print(f'{i} v.',i.__version__)
        except:
            print(i)

ver_check()

In [93]:
# import_list = list(dir())
# for i in import_list:
#     print(i.__version__)

In [94]:
### 데이터로딩
DATA_FILE = r'C:\Git\KDT\BigData\data\iris.csv'

### CSV => DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

### 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
 - 클래스목적 : iris 데이터를 학습 및 추론 목적 
 - 클래스이름 : IrisRegModel
 - 부모클래스 : nn.Module
 - 매개__변수 : 층별 입출력 개수 고정하기 때문에 필요없음!
 - 속성__필드 : featureDF, targetDF, n_rows, n_features 
 - 기능  역할 : __init__()  : 모델 구조 설정, forward() : 순방향 학습, <= 오버라이딩(overriding)
 - 클래스구조 
    * 입력층 : 입력 3개 (피쳐)   출력 10개 (퍼셉트론 / 뉴런 10개 존재)
    * 은닉층 : 입력 10개         출력 30개 (퍼셉트론 / 뉴런 30개 존재)
    * 출력층 : 입력 30개         출력 1개 (너비값)


 - 활성화함수
   * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드
   * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [95]:
class IrisRegModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)


    def forward(self, input_data):
        y = self.in_layer(input_data)   # f11w11 + f12w12 + f13w13 ... + f103w103 + b
        # 여러 정보를 저장하는 배열
        # 활성화 함수에 들어가야지?
        y = F.relu(y)                       # relu => y 값의 범위 : 0 <= y  // 시그모이드는 0 ~ 1이긴 해.

        # 10개의 숫자를 갖고 은닉층 출동
        # - 은닉층 : 10개의 숫자 값 (>= 0)
        y = self.hidden_layer(y)                # f21w11 + f22+w12 + f23w13 ... f210w210 + b , f230w201 + f230w202 ... f230w230+b  
        y = F.relu(y)                       # 2번의 1번부터 시작, 2번의 30번까지.

        # - 출력층 : 30개의 숫자 값 (>=0)
        return self.out_layer(y)                   # f31w31 .... f330w330 + b   // 3번의 30번
        # 회귀이므로 바로 반환        

In [96]:
### 모델 인스턴스
model = IrisRegModel()



In [97]:
### 모델 사용 메모리 정보 확인
summary(model, input_size=(10,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

[1] 데이터셋 클래스 설계 및 정의 <hr>
 - 데이터셋 : iris.csv
 - 피쳐_개수 : 3개
 - 타겟_개수 : 1개
 - 클래스이름 : IrisDataset
 - 부모클래스 : utils.data.Dataset
 - 속성_필드 : featureDF, targetDF, n_rows, n_features
 - 필수 메서드
    * __init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * __len__(self) : 데이터 개수 반환
    * __getitem__(self, index): 특정 인덱스의 피쳐와 타겟 반환

In [98]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
        
    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화

        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # 그대로 가져오면 시리즈니까 values로 형 변환
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 변환

        return featureTS, targetTS

In [99]:
## 데이터셋 인스턴스 생성
# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150,1)
targetDF = irisDF[irisDF.columns[-1:]]  # 2D (150,1)

# 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

[4] 학습 준비
 - 학습 횟수 : EPOCH            <- 처음 ~ 끝까지 공부하는 단위
 - 배치 크기 : BATCH_SIZE       <- 한번에 학습할 데이터셋 양
 - 위치 지정 : DEVICE           <- 텐서 저장 및 실행 위치 (GPU / CPU)
 - 학_습_률 : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1

In [100]:
### 학습 진행 관련 설정
EPOCH = 100
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]/BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

# print(f'BATCH_CNT : {BATCH_CNT}')

 - 인스턴스 / 객체 : 모델, 데이터셋, 최적화 (, 손실함수 , 성능지표)

In [101]:
# 모델 인스턴스
model = IrisRegModel()

# # 데이터셋 인스턴스
irisDS = IrisDataset(featureDF,targetDF)

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)


trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test,y_test)

trainDL = DataLoader(irisDS, batch_size=BATCH_SIZE)



# # 데이터로더 인스턴스
irisDL = DataLoader(irisDS, batch_size=BATCH_SIZE)

In [102]:
irisDL.dataset.n_features, irisDL.dataset.n_rows

(3, 150)

In [103]:
## [테스트] 데이터로더
for feature, target in irisDL:
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [104]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(), lr=LR)

# 최적화 스케쥴링 인스턴스 생성 => lr 조절 및 성능 개선 여부 체크
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', patience =5, verbose = True)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE ....
reqloss = nn.MSELoss()


c:\Users\kjy19\anaconda3\envs\TORCH_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[5] 학습 진행

In [105]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = irisDS.n_rows/BATCH_SIZE
print(f'CNT => {CNT}')

## 학습 모니터링 / 스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT = 0

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    score_total, loss_total = 0,0
    for featureTS, targetTS in irisDL:
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = reqloss(pre_y, targetTS)
        loss_total += loss.item()
        # 성능평가 개선
        score = R2Score()(pre_y,targetTS)
        score_total += score.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():

        # 에포크 당 손실값과 성능평가값 저장
        # 검증 데이터셋 
        # 일부 뽑기 ....을마나 뽑을까 ... ㅎㅎ,ㅔ헤헤ㅔ
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val = model(val_featureTS)
        # 손실
        loss_val = reqloss(pre_val, val_targetTS)
        # 성능평가
        score_val = R2Score()(pre_val, val_targetTS)


    # 최적화 스케쥴러 인스턴스 업데이트
    # scheduler.step(loss_val)  # 최소값이 좋은 것.
    scheduler.step(score_val)   # 최댓값 해야함 scheduler에 모드 max 추가.
    
    print(f'scheduler.num_bad_epochs {scheduler.num_bad_epochs}', end=" ")
    print(f'scheduler.patience {scheduler.patience}')
    # 손실 감소 (또는 성능 개선)이 안되는 경우 조기종료
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f'{scheduler.patience}EPOCH 성능 개선이 없어 조기종료함')
        break

CNT => 15.0
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 scheduler.patience 5
scheduler.num_bad_epochs 0 s

In [106]:
val_featureTS = torch.FloatTensor(valDS.featureDF.values)
val_targetTS = torch.FloatTensor(valDS.targetDF.values)

val_featureTS.shape, val_targetTS.shape

(torch.Size([28, 3]), torch.Size([28, 1]))

In [107]:
print(f'CNT : {CNT}')

CNT : 15.0


In [108]:
print(f'LOSS HISTORY => {LOSS_HISTORY}')
print(f'SCORE HISTORY => {SCORE_HISTORY}')


LOSS HISTORY => [[], []]
SCORE HISTORY => [[], []]


 - 모델 저장 방법 <hr>
 * 방법1 : 모델 파라미터만 저장
 * 방법2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [109]:
### 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[ 0.5431,  0.4900, -0.4409],
                      [ 0.2134, -0.3632,  0.6082],
                      [-0.2089, -0.2161, -0.4890],
                      [-0.2563,  0.0673,  0.3008],
                      [-0.1674, -0.0349, -0.5671],
                      [ 0.5713,  0.3292, -0.0136],
                      [-0.3658,  0.2943,  0.0271],
                      [ 0.0862,  0.4203, -0.3746],
                      [-0.3287,  0.1831, -0.4099],
                      [ 0.4205, -0.0985, -0.1420]])),
             ('in_layer.bias',
              tensor([ 0.5687, -0.4836,  0.3944, -0.4592,  0.0905, -0.4817,  0.3690,  0.4837,
                       0.2955, -0.1898])),
             ('hidden_layer.weight',
              tensor([[ 2.6192e-01,  1.2231e-01, -2.7211e-01, -9.0293e-02,  3.8885e-02,
                        1.1733e-02, -1.8474e-01,  3.6989e-02,  8.5088e-02, -1.0200e-01],
                      [-2.7434e-02, -2.1623e-01, -2.8044e-02,  1.5134e-0

 - [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [110]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris'

# 저장 파일명
SAVE_FILE = 'model_train_wbs.pth'

In [111]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)  # 폴더 / 폴더 / ... 하위 폴더까지 생성

In [112]:
# 모델 저장
torch.save(model.state_dict(), SAVE_PATH + SAVE_FILE)

In [113]:
# 로딩으로 가중치 파일만 들고가서 다른 곳에서 같은 환경에서 그대로 돌리기 가능

# 학습 된 것만 가져가서 쓰기도 간으

# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
# [2] 모델의 State_dict 속성에 저장
# [3]

# 읽기
wbTS = torch.load(SAVE_PATH+SAVE_FILE, weights_only=True)
print(type(wbTS))

<class 'collections.OrderedDict'>


In [114]:
# 모델 인스턴스에 저장 
model.load_state_dict(wbTS)

<All keys matched successfully>

In [115]:
model2 = irisRegModel() # W,b 초기화

model.load_state_dict(wbTS)

NameError: name 'irisRegModel' is not defined